# Prediction of TF binding based on DNA physical properties
COMP561: Computational Biology Methods and Research

### About:
A method to predict TF binding sites based on DNA physical properties.
### Why I chose this project
- I chose to do this project (came from within vs being imposed from outside)
- There were so many things that I didn't know how to do and I faced many issues (e.g. running time). To solve them, I explored and found out solutions myself, in the forms of Python package, way to efficiently run a tool (Galaxy), and so on. It was an assignment, but I acted very proactively. I learned a lot and had a lot of fun.
- I was very immersed when I was doing it: Without noticing, I was working for hours in a row and time flew by. 

### DNA physical properties can affect TF binding
Structural properties of DNA have a significant impact on DNA's affinity for proteins
- e.g. minor groove width (MGW), Roll, helical twist (HelT), and propeller twist (ProT)


DNA physical properties can be predicted from the local DNA sequence using tools such as DNAshape: https://academic.oup.com/nar/article/43/D1/D103/2439587

### Identification of binding sites for a given TF is important
- e.g. Application: structure-based drug discovery

### ChIP-seq combined with position weight matrix (PWM) can be used to identify TF binding sites
- Issue: Sites on DNA that look like they should be bound by TFs but are not in reality (=negative examples)
- positive examples = sequences in which TFs are bound

### Objective: To examine whether positive examples could be distinguished from negative examples based on predicted structural properties of their sequences

### Physical properties used
- Minor groove width (MGW)
- Roll
- Helical twist (HelT)
- Propeller twist (ProT)

### TFs used: TATA box-binding protein (TBP)
Rationales
- TBP: One of the most well-known and essential TF. Required for correct initiation of transcription and is used by all three eukaryotic RNA polymerases

### Machine learning models used:
- K-nearest neighbour (KNN)
- Logistic regression

### Project Overview: Steps involved
1. Identifying a set of bound and non-bound DNA sequences for a given TF (based on existing experimental data)
2. Obtaining the DNA physical properties of each sequence
3. Training machine learning classifiers to distinguish between bound and unbound sites.

## 1. Identification of a set of bound and non-bound DNA sequences for a given TF 
Based on existing experimental data

In [1]:
# If pyranges is not installed
#import sys
#!{sys.executable} -m pip install pyranges

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import pyranges as pr
import time

In [3]:
overall_runtime_start = time.time()

In [4]:
# Create a dataframe from the textfile
file_name = "factorbookMotifPos.txt"  # Contains genomic coordinates of positive examples of the target TFs
df = pd.read_csv(file_name, usecols=[1,2,3,4,5,6], 
                 names=['Chom', 'TFStart', 'TFEnd', 'TFName', 'ScoreBS', 'StrandBS'],
                 header=None,
                 sep='\t')

# Get an overview of the dataframe
print(df.shape)
print(df.head(10))

(2366151, 6)
   Chom  TFStart   TFEnd TFName  ScoreBS StrandBS
0  chr1    10461   10476    UA1     2.02        -
1  chr1    10464   10479    UA1     2.26        +
2  chr1    16245   16260   CTCF     1.97        -
3  chr1    89933   89948    NFY     1.95        -
4  chr1    91265   91280   CTCF     1.40        +
5  chr1    91419   91434   CTCF     2.07        +
6  chr1    91421   91436   CTCF     2.80        -
7  chr1    91431   91446   CTCF     1.95        -
8  chr1   104986  105001   CTCF     2.83        +
9  chr1   138972  138987   CTCF     3.43        -


In [5]:
# List out all the transcription factors (TFs)
df_TFs = df["TFName"].value_counts()
print(df_TFs)
df_TFs.to_csv("TFs.csv")  # Create a csv file containing all the TFs
print("\nNumber of types of transcription factors:", df_TFs.shape[0])

AP1       166122
CTCF      158004
UAK42     130536
SP1        83782
ZNF263     75577
           ...  
UA14         149
NR4A1         90
UAK32         21
UAK44          8
UAK53          6
Name: TFName, Length: 133, dtype: int64

Number of types of transcription factors: 133


## Extract relevant rows: the TF x the positive examples
The + and - strands in the bedfile is unknown; therefore, will deal only with + strand.

In [6]:
# Specify the TF
tf = 'TBP'  # 'UA7' 'AP2'

In [7]:
# Extract rows containing TBP binding sites information
df_TF = df[df['TFName'] == tf]
df_TF_fwd = df_TF[df_TF['StrandBS'] == '+']
print(df_TF_fwd.shape)
print(df_TF_fwd.head(10))

(392, 6)
       Chom   TFStart     TFEnd TFName  ScoreBS StrandBS
4756   chr1   1850728   1850748    TBP     0.79        +
14974  chr1   8086379   8086399    TBP     1.14        +
14975  chr1   8086381   8086401    TBP     1.32        +
23746  chr1  12538132  12538152    TBP     1.25        +
24187  chr1  12717648  12717668    TBP     1.39        +
30749  chr1  17575548  17575568    TBP     1.34        +
40367  chr1  23886002  23886022    TBP     1.13        +
48368  chr1  27339350  27339370    TBP     1.20        +
48743  chr1  27481630  27481650    TBP     1.43        +
52121  chr1  28908396  28908416    TBP     1.74        +


## Read the bed file

In [8]:
# The bed file: contains active regulatory regions in GM12879 (a cell line derived from lymphoblasts)
gr = pr.read_bed("wgEncodeRegTfbsClusteredV3.GM12878.merged.bed")
df_bed = gr.df
print(df_bed.shape)
print("Total # of active regulatory regions:", df_bed.shape[0], "\n")

# Observe the top and bottom of the bed file
print(df_bed.head(10))
print(df_bed.tail(10))

(78792, 4)
Total # of active regulatory regions: 78792 

  Chromosome   Start     End     Name
0       chr1  237550  237989   chr1.1
1       chr1  521310  521756   chr1.2
2       chr1  713702  714675   chr1.3
3       chr1  762022  763345   chr1.4
4       chr1  805100  805473   chr1.5
5       chr1  839802  841023   chr1.6
6       chr1  848092  848620   chr1.7
7       chr1  856280  856826   chr1.8
8       chr1  858297  862058   chr1.9
9       chr1  873457  874094  chr1.10
      Chromosome      Start        End       Name
78782       chrX  154764452  154764876  chrX.2507
78783       chrX  154807318  154807679  chrX.2508
78784       chrX  154841965  154843013  chrX.2509
78785       chrX  154880827  154881309  chrX.2510
78786       chrX  154891942  154892376  chrX.2511
78787       chrX  155049813  155050189  chrX.2512
78788       chrX  155087251  155087687  chrX.2513
78789       chrX  155094268  155094730  chrX.2514
78790       chrX  155110578  155111395  chrX.2515
78791       chrX  1551965

In [9]:
# List out all chromosomes
list_choms = list(set(df_bed["Chromosome"].to_list()))
print(sorted(list_choms))
"""The list was created after inspecting the chromosomes contained in the bed file.
The list of chromosomes were created as below for ordering (sorting the list of chromosomes generated directly from the bed
file dataframe resulted in an undesirable ordering of chromosomes)"""
print("-------------")
list_chrs = ['chr%s' % s for s in range(1,23)]
list_chrs.append("chrX")
print(list_chrs)

['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr2', 'chr20', 'chr21', 'chr22', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chrX']
-------------
['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX']


### Extracting negative examples
Extracting genomic coordinates of sequences potentially containing negative examples

In [18]:
## Before the edit on df_TF_negative
start_time_total = time.time()
df_TF_negative_all = pd.DataFrame(columns=['Chom','NStart','NEnd','UnboundTF'])  # An empty dataframe used later

for chom in list_chrs:  # For each chromosome (chr)
    start_time = time.time()
    print(chom)

    # Extract portion of the bed file corresponding to the chromosome
    df_bed_chom = df_bed[df_bed["Chromosome"] == chom]
    # print(df_bed_chom.head(10))
    # Extract portion of the genomic coordinate (of +'ve example) df corresponding to the chr
    df_TF_chom = df_TF_fwd[df_TF_fwd["Chom"] == chom]
    #print(df_TF_chom.head(10))
    
    # Sorting to ensure the dataframe is sorted
    df_bed_chom_s = df_bed_chom.sort_values("Start")
    df_TF_chom_s = df_TF_chom.sort_values("TFStart")

    
    # For reducing running time
    # Get range for the coordinates of active regulatory regions
    min_bed = df_bed_chom_s["Start"].min()
    max_bed = df_bed_chom_s["End"].max() 

   # Get range for the coordinates of TF bound sites (positive examples)
    min_TF = df_TF_chom_s["TFStart"].min()
    max_TF = df_TF_chom_s["TFEnd"].max()

    """
    Idea:
    Negative examples will fall under the active regulatory region.
    => Potential negative examples will be regions in active regulatory regions that 
    doesn't loverlap with locations of positive examples
    i.e. Negative examples can't possibly be anywhere in the active regulatory region
    that overlaps with positive examples
    
    min active      minTF                 maxTF            max active
    
    |=================|----------------------|===============|
    """
    # Extract all the coordinates of active regulatory region that comes before the 
    # smallest coordinate among the positive examples for the chromosome
    df_bed_non_top = df_bed_chom_s[df_bed_chom_s["End"] < min_TF]
    df_bed_non_top = df_bed_non_top.rename(columns = {"Chromosome":"Chom", "Start":"NStart", "End":"NEnd"})
    if df_bed_non_top.empty == False:
        df_bed_non_top.loc[:, "UnboundTF"] = tf
    df_bed_non_top = df_bed_non_top.drop(["Name"], axis=1)

    # Extract all the coordinates of active regulatory region that comes after the 
    # largest coordinate among the positive examples for the chromosome
    df_bed_non_bottom = df_bed_chom_s[max_TF < df_bed_chom_s["Start"]] # need to append these
    df_bed_non_bottom = df_bed_non_bottom.rename(columns = {"Chromosome":"Chom", "Start":"NStart", "End":"NEnd"})
    if df_bed_non_bottom.empty == False:
        df_bed_non_bottom.loc[:, "UnboundTF"] = tf
    df_bed_non_bottom = df_bed_non_bottom.drop(["Name"], axis=1)
    
    
    """
    min active      minTF                 maxTF            max active
    
    |=================|----------------------|===============|
    """
    # Extract coordinates of active regulatory region that may overlap with coordinates of positive examples
    df_bed_relev = df_bed_chom_s[df_bed_chom_s["Start"] <= max_TF]
    df_bed_relev = df_bed_relev[min_TF <= df_bed_relev["End"]]
    print("dimension of bed_relev", df_bed_relev.shape)
    
    
    # Identify potential regative examples
    list_cols_TF = list(df_TF_chom_s.columns.values)
    df_TF_negative = pd.DataFrame(columns=['Chom','NStart','NEnd','UnboundTF'])  # Create an empty data frame

    count = 0
    for index_b, row_b in df_bed_relev.iterrows(): # For each active regulatory region (shortlisted)
        X = row_b["Start"]
        Y = row_b["End"]

        df_TF_Inregion = pd.DataFrame(columns=list_cols_TF)  # Create an empty dataframe
        # Gather any positive examples that may fall within the active regulatory region
        """
        X                           Y
        |======aaaa===aaaa====aaaa==|
        """
        for index_TF, row_TF in df_TF_chom_s.iterrows():
            start = row_TF['TFStart'] 
            end = row_TF['TFEnd']    
            if X <= start and end <= Y:  # If the coordinates of positive examples fall within the active regulatory region
                df_1row = df_TF_chom_s.loc[[index_TF]]
                df_TF_Inregion = df_TF_Inregion.append(df_1row)

        # If at least one positive example falls under the active regulatory region
        if df_TF_Inregion.empty == False:
            # print(count)
            df_TF_Inregion_r = df_TF_Inregion.reset_index()

            idx_count = df_TF_Inregion_r.shape[0]

            for index_InR, row_InR in df_TF_Inregion_r.iterrows():
                st = row_InR["TFStart"]
                ed = row_InR["TFEnd"]

                if index_InR == 0:  # If it's the first row
                    new_row1 = {'Chom': chom, 'NStart': X, 'NEnd': st-1, 'UnboundTF': tf}
                    df_TF_negative = df_TF_negative.append(new_row1, ignore_index=True)

                if index_InR == idx_count - 1: # If it's the last row
                    new_row2 = {'Chom': chom, 'NStart': ed+1, 'NEnd': Y, 'UnboundTF': tf}
                    df_TF_negative = df_TF_negative.append(new_row2, ignore_index=True)
                    break
                else:  # If there are multiple binding sites within a region and if the row is not the last row
                    st_2 = int(df_TF_Inregion_r.loc[[index_InR+1]].TFStart)
                    ed_2 = int(df_TF_Inregion_r.loc[[index_InR+1]].TFEnd)
                    if st_2 > ed: # If the TF binding sites don't overlap
                        new_row = {'Chom': chom, 'NStart': ed+1, 'NEnd': st_2-1, 'UnboundTF': tf}
                        df_TF_negative = df_TF_negative.append(new_row, ignore_index=True)

        count += 1    
    
    # Create a dataframe that contains all the negative examples
    """
    min active      minTF                      maxTF            max active
    |=================|======aaaa===aaaa====aaaa==|===============|
    """
    all_negative_dfs = [df_bed_non_top, df_TF_negative, df_bed_non_bottom]
    df_negative_all = pd.concat(all_negative_dfs).reset_index(drop=True)
    print(df_negative_all)
    end_time = time.time()
    run_time = end_time - start_time
    print("Running time for {}: {}".format(chom, run_time))
    
    frames = [df_TF_negative_all, df_negative_all]
    df_TF_negative_all = pd.concat(frames).reset_index(drop=True)
    
    print("====================")

end_time_total = time.time()
run_time_total = end_time_total - start_time_total
print(df_TF_negative_all)
print(df_TF_negative_all.shape)
print("Total running time: ", run_time_total)

chr1
dimension of bed_relev (7110, 4)
     Chom     NStart       NEnd UnboundTF
0    chr1     237550     237989       TBP
1    chr1     521310     521756       TBP
2    chr1     713702     714675       TBP
3    chr1     762022     763345       TBP
4    chr1     805100     805473       TBP
..    ...        ...        ...       ...
319  chr1  249157159  249158079       TBP
320  chr1  249167142  249168812       TBP
321  chr1  249199877  249201223       TBP
322  chr1  249206342  249206773       TBP
323  chr1  249218925  249219295       TBP

[324 rows x 4 columns]
Running time for chr1: 29.52905011177063
chr2
dimension of bed_relev (5929, 4)
     Chom     NStart       NEnd UnboundTF
0    chr2      11419      12069       TBP
1    chr2      18635      19084       TBP
2    chr2     142099     142535       TBP
3    chr2     172020     172680       TBP
4    chr2     246471     246895       TBP
..    ...        ...        ...       ...
146  chr2  243027932  243028626       TBP
147  chr2  24303018

      Chom     NStart       NEnd UnboundTF
0    chr14   20264672   20265133       TBP
1    chr14   20346179   20346611       TBP
2    chr14   20415835   20416217       TBP
3    chr14   20612067   20612495       TBP
4    chr14   20664051   20664481       TBP
..     ...        ...        ...       ...
519  chr14  107256518  107256971       TBP
520  chr14  107259096  107260261       TBP
521  chr14  107280756  107281564       TBP
522  chr14  107283692  107284122       TBP
523  chr14  107287341  107287792       TBP

[524 rows x 4 columns]
Running time for chr14: 5.929144620895386
chr15
dimension of bed_relev (1375, 4)
       Chom     NStart       NEnd UnboundTF
0     chr15   20398188   20398853       TBP
1     chr15   20561391   20562017       TBP
2     chr15   21183357   21184021       TBP
3     chr15   21927229   21927614       TBP
4     chr15   22387307   22388087       TBP
...     ...        ...        ...       ...
1177  chr15  102215983  102216558       TBP
1178  chr15  102264074  102

In [14]:
df_TF_negative_fwd = df_TF_negative_all
df_TF_negative_all_fwd = df_TF_negative_all.reindex(columns= ['Chom', 'NStart', 'NEnd','UnboundTF', 'Strand'])
df_TF_negative_all_fwd.loc[:, "Strand"] = "+"  # Extract the forward strand
print(df_TF_negative_all_fwd.head(10))
print(df_TF_negative_all_fwd.shape)

   Chom  NStart    NEnd UnboundTF Strand
0  chr1  237550  237989       TBP      +
1  chr1  521310  521756       TBP      +
2  chr1  713702  714675       TBP      +
3  chr1  762022  763345       TBP      +
4  chr1  805100  805473       TBP      +
5  chr1  839802  841023       TBP      +
6  chr1  848092  848620       TBP      +
7  chr1  856280  856826       TBP      +
8  chr1  858297  862058       TBP      +
9  chr1  873457  874094       TBP      +
(9309, 5)


In [ ]:
df_all = df_TF_negative_all

In [ ]:
# Save the result (genomic coordinates of potential negative examples on forward strand) in a csv file
fname = tf + "_negative_fwd.csv"
df_TF_negative_all_fwd.to_csv(fname, index=False)

In [ ]:
# Save the result in a bed file
print(df_TF_negative_all_fwd.head(10))
df_TF_negative_all_fwd = df_TF_negative_all_fwd.drop(["UnboundTF", "Strand"], axis=1)
print(df_TF_negative_all_fwd.head(10))
fOutName = tf + "_fwd_neg.bed"
df_TF_negative_all_fwd.to_csv(fOutName, index=False, header=False, sep="\t")

In [ ]:
# Check that the bed file was successfully created
gr2 = pr.read_bed(fOutName)
df2_bed = gr2.df
print(df2_bed.shape)
print(df2_bed.head(5))

## 2. Obtaining DNA physical properties of each sequence
GB Shape: http://rohsdb.cmb.usc.edu/
"Predicted structural properties for every human genome positions are available here"

### Reminder: Goal of the project
To determine whether the sites that are bound (positive examples) can be distinguished from those that are not bound (negative example) based on predicted physical properties of the sequence

In [ ]:
# Create textfiles that can be directly used as input for DNAshape
# Positive examples
df_TF_fwd_m = df_TF_fwd.drop(["TFName", "ScoreBS", "StrandBS"], axis=1)
print(df_TF_fwd_m.head(10))
fout_name = "factorbookMotifPos_GBshape_" + tf
df_TF_fwd_m.to_csv(fout_name + ".txt", sep="\t", index=False, header=False)
df_TF_fwd_m.to_csv(fout_name + ".bed", sep="\t", index=False, header=False)

In [ ]:
overall_runtime_end = time.time()
overall_runtime = overall_runtime_end - overall_runtime_start
print("Overall running time (s):", overall_runtime)

### 2.1: Extract, from hg19, the actual sequences of bound and unbound sites.

### Extract sequences of positive examples and potential negative examples: Using Galaxy
Workflow
1. For each fasta file of a chromosome (e.g. chr1)
2. Run Galaxy's "Extract Genomic DNA" using the fasta file and "factorbookMotifPos_TBP_fwd_pos.bed" which contains the positions of positive examples (where TFs are actually bound)
3. Download the output
4. Rename the output as, "seqs_chr#_TBP_fwd_pos.fasta" (e.g. seqs_chr1_TBP_fwd_pos.fasta)
5. Repeat for all chromosome
6. Repeat 1-5 for negative example. Change file names as necessary.